<a href="https://colab.research.google.com/github/VGGatGitHub/Summer2021/blob/main/D-Wave_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Notebook experimenting how to use DOcplex and IBM Quantum.

# 0. Install libraries & clone repo

In [ ]:
import sys

if hasattr(sys, 'real_prefix'):
    #we are in a virtual env.
    !pip install cplex && pip install docplex
else:
    !pip install --user cplex && pip install --user docplex

print()
!pip show cplex
print()
!pip show docplex

#it my be useful to retstart you runtime

In [ ]:
!pip install qiskit
!pip show qiskit

# !pip install qiskit-optimization
#!pip show qiskit-optimization

#it my be useful to retstart you runtime

**Possibly need to restart runtime here.**

In [ ]:
#it my be useful to retstart you runtime

# clone repo
!git clone https://github.com/VGGatGitHub/Summer2021.git

Cloning into 'Summer2021'...
remote: Enumerating objects: 112, done.
remote: Counting objects: 100% (112/112), done.
remote: Compressing objects: 100% (98/98), done.
remote: Total 112 (delta 52), reused 19 (delta 10), pack-reused 0
Receiving objects: 100% (112/112), 205.01 KiB | 6.41 MiB/s, done.
Resolving deltas: 100% (52/52), done.


# 1. Load Data

In [83]:
#next run the model.py in the folder introScenario
!pwd
%cd ./Summer2021/introScenario/
#./Summer2021/intermediateScenario
#./Summer2021/introScenario/
#%ls


/content/Summer2021/introScenario
[Errno 2] No such file or directory: './Summer2021/introScenario/'
/content/Summer2021/introScenario


In [84]:
import pandas as pd

# supply + costs
supply_file = './plants.csv'

# demand
demand_file = './customerDemand.csv'

# for now load example data (plants.csv and customerDemand.csv under introScenario folder)
s_df = pd.read_csv(supply_file)
d_df = pd.read_csv(demand_file)

In [85]:
d_df

,Product,Demand
0,handSanitizer,100
1,mask,120


In [86]:
s_df

,Plants,Cost,Capacity,Product
0,1,3,40,mask
1,2,2,30,mask
2,3,1,30,handSanitizer
3,4,3,100,handSanitizer
4,5,2,60,mask
5,6,1,45,mask


In [88]:
inputs={'customerDemand':d_df,'plants':s_df}

# 2. Solve the DO problem via DOcplex

In [89]:
#If you get an error you may have to restart the runtime
import cplex
import docplex.mp
from docplex.mp.model import Model

In [90]:
global output_lock
#with output_lock:

try:  
  if output_lock:
    print(output_lock)
except:
  output_lock=dict({'output_lock':{}})

try: 
  if outputs:
    print(outputs)
except:
  outputs=dict()


{'output_lock': {}}
{'plantsAllocation':    plants  ...  plants minValueAllocationForAssignment
0       1  ...                                       0
1       2  ...                                       0
2       3  ...                                       0
3       4  ...                                       0
4       5  ...                                       0
5       6  ...                                       0

[6 rows x 7 columns]}


In [91]:
exec(open("./model.py").read())

* building wado model
model building done in 0.10219979286193848 secs
* running wado model
Version identifier: 20.1.0.0 | 2020-11-11 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
CPXPARAM_TimeLimit                               120
Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 10 rows and 6 columns.
All rows and columns eliminated.
Presolve time = 0.01 sec. (0.01 ticks)

Root node processing (before b&c):
  Real time             =    0.01 sec. (0.01 ticks)
Parallel b&c, 2 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.01 sec. (0.01 ticks)
Solve status: JobSolveStatus.OPTIMAL_SOLUTION
* model docplex_model3 solved with objective = -58080.000
*  KPI: total plants allocations                  = 61440.000
*  KPI: total Cost of plants over all alloca

In [92]:
outputs


{'plantsAllocation':    plants  ...  plants minValueAllocationForAssignment
 0       1  ...                                       0
 1       5  ...                                       0
 2       6  ...                                       0
 
 [3 rows x 7 columns]}

In [93]:
outputs['plantsAllocation']

,plants,plants allocation decision,plants selection decision,plants Capacity,plants Product,plants Cost,plants minValueAllocationForAssignment
0,1,15.0,1.0,40,mask,3,0
1,5,60.0,1.0,60,mask,2,0
2,6,45.0,1.0,45,mask,1,0


In [94]:
model

docplex.mp.Model['docplex_model3']

In [95]:
solution=model.solve()
print(solution)
solution

solution for: docplex_model3
objective: -58080
x1=15
x2=60
x3=45
x4=1
x5=1
x6=1



docplex.mp.solution.SolveSolution(obj=-58080,values={x1:15,x2:60,x3:45,x..

In [96]:
cost = solution.get_objective_value()
x = solution.get_all_values()
x = np.array(x)
print(cost,x)

-58080.0 [15. 60. 45.  1.  1.  1.]


# 3. Solve the problem using QISKit

In [ ]:
# take a look at https://quantum-computing.ibm.com/lab/docs/iql/optimization

#from qiskit_optimization import QuadraticProgram
#from qiskit_optimization.translators import from_docplex_mp

from qiskit.optimization import QuadraticProgram

# load from a Docplex model
mod = QuadraticProgram()
mod.from_docplex(model)
print(mod.export_as_lp_string())

In [ ]:
print('constant:\t\t\t', mod.objective.constant)

print('linear dict:\t\t\t', mod.objective.linear.to_dict())
print('linear array:\t\t\t', mod.objective.linear.to_array())
print('linear array as sparse matrix:\n', mod.objective.linear.coefficients, '\n')

print('quadratic dict w/ index:\t', mod.objective.quadratic.to_dict())
print('quadratic dict w/ name:\t\t', mod.objective.quadratic.to_dict(use_name=True))
print('symmetric quadratic dict w/ name:\t', mod.objective.quadratic.to_dict(use_name=True, symmetric=True))
print('quadratic matrix:\n', mod.objective.quadratic.to_array(),'\n')
print('symmetric quadratic matrix:\n', mod.objective.quadratic.to_array(symmetric=True),'\n')
print('quadratic matrix as sparse matrix:\n', mod.objective.quadratic.coefficients)


In [ ]:
%time result = mod.solve()
print("x={}".format(result.x))
print("fval={:.2f}".format(result.fval))

In [ ]:
from qiskit.optimization.converters import QuadraticProgramToQubo
conv = QuadraticProgramToQubo()
mod_Qubo = conv.convert(mod)


In [ ]:
print('constant:\t\t\t', mod_Qubo.objective.constant)
print('linear array as sparse matrix:\n', mod_Qubo.objective.linear.coefficients, '\n')

print('quadratic matrix as sparse matrix:\n', mod_Qubo.objective.quadratic.coefficients)

In [ ]:
#exact, qaoa, rqaoa, grover_optimizer, admm
%time result = qaoa.solve(mod) #rqaoa, grover_optimizer run out of memory after 28min

print(result)
#statevector_simulator with 402 qubits for admm.solve(mod) and admm_rqaoa had filed after 4m